# Neruo Kit
- https://neuropsychology.github.io/NeuroKit/functions/ecg.html

In [1]:
import os
import pandas as pd
import neurokit2 as nk
import numpy as np

ecg_path = '/tf/hsh/new_ecg/lead2/'
ecg_list = os.listdir(ecg_path)

# Load the ECG data
data = pd.read_csv(f"{ecg_path}/{ecg_list[2312]}")

In [2]:
# Extract the II lead signal and sampling rate
ecg_signal = data["II"].values
sampling_rate = 500  # Assuming 500Hz sampling rate based on the time steps in your data

# 1. Preprocess the ECG signal
ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=sampling_rate)

# 2. Extract ECG features
ecg_signals, ecg_info = nk.ecg_process(ecg_cleaned, sampling_rate=sampling_rate)

# Extract R-peaks (for rhythm analysis and heart rate calculation)
r_peaks = ecg_info.get('ECG_R_Peaks', [])

# Heart Rate and Rhythm Calculation
if len(r_peaks) > 1:
    rr_intervals = np.diff(r_peaks) / sampling_rate  # Time difference between successive R peaks (in seconds)
    heart_rate = 60 / np.mean(rr_intervals)  # Convert RR interval to beats per minute (bpm)
    rhythm = 'regular' if np.std(rr_intervals) < 0.12 else 'irregular'
else:
    heart_rate = np.nan
    rhythm = "unknown"

# P-waves detection and PR Interval calculation
p_peaks = ecg_info.get('ECG_P_Peaks', [])
pr_intervals = []
if len(p_peaks) > 0 and len(r_peaks) > 0:
    for p_peak in p_peaks:
        r_after_p = r_peaks[r_peaks > p_peak]
        if len(r_after_p) > 0:
            pr_interval = (r_after_p[0] - p_peak) / sampling_rate
            pr_intervals.append(pr_interval)
mean_pr_interval = np.mean(pr_intervals) if len(pr_intervals) > 0 else np.nan

# QRS Complex duration calculation (handling nan values)
qrs_onsets = ecg_info.get('ECG_R_Onsets', [])
qrs_offsets = ecg_info.get('ECG_R_Offsets', [])
qrs_durations = []
for onset, offset in zip(qrs_onsets, qrs_offsets):
    if not np.isnan(onset) and not np.isnan(offset):
        qrs_duration = (offset - onset) / sampling_rate
        qrs_durations.append(qrs_duration)
mean_qrs_duration = np.mean(qrs_durations) if len(qrs_durations) > 0 else np.nan

# T-wave detection and QT Interval calculation (handling nan values)
t_peaks = ecg_info.get('ECG_T_Peaks', [])
qt_intervals = []
for onset, t_peak in zip(qrs_onsets, t_peaks):
    if not np.isnan(onset) and not np.isnan(t_peak):
        qt_interval = (t_peak - onset) / sampling_rate
        qt_intervals.append(qt_interval)
mean_qt_interval = np.mean(qt_intervals) if len(qt_intervals) > 0 else np.nan

# ST Segment Analysis
st_segment_window = int(0.08 * sampling_rate)  # 80 ms window
st_elevations = []
if len(qrs_offsets) > 0:
    for offset in qrs_offsets:
        if offset + st_segment_window < len(ecg_cleaned):
            st_elevation = np.mean(ecg_cleaned[offset:offset + st_segment_window])
            st_elevations.append(st_elevation)
mean_st_elevation = np.mean(st_elevations) if len(st_elevations) > 0 else np.nan

# T-wave analysis: Inverted T-waves
t_peaks = [int(t) for t in t_peaks if not np.isnan(t)]
if len(t_peaks) > 0:
    t_wave_amplitudes = ecg_cleaned[t_peaks]
    t_wave_inverted = np.any(t_wave_amplitudes < 0)
else:
    t_wave_inverted = False

# 결과 출력
print("Heart Rate (bpm):", heart_rate)
print("Rhythm:", rhythm)
print("PR Interval (mean, seconds):", mean_pr_interval)
print("QRS Duration (mean, seconds):", mean_qrs_duration)
print("QT Interval (mean, seconds):", mean_qt_interval)
print("ST Segment Elevation (mean):", mean_st_elevation)
print("T-Wave Inverted:", t_wave_inverted)


Heart Rate (bpm): 90.81735620585268
Rhythm: regular
PR Interval (mean, seconds): 0.14542857142857143
QRS Duration (mean, seconds): 0.2424
QT Interval (mean, seconds): 0.3652
ST Segment Elevation (mean): -0.005848767609509961
T-Wave Inverted: False


In [ ]:
print(ecg_info['ECG_R_Onsets'],ecg_info['ECG_R_Offsets'])

In [ ]:
print(qrs_onsets, qrs_offsets)

In [ ]:
t_peaks